# 猫和老鼠

https://leetcode.cn/problems/cat-and-mouse?envType=daily-question&envId=2025-02-10

两位玩家分别扮演猫和老鼠，在一张 无向 图上进行游戏，两人轮流行动。

图的形式是：graph[a] 是一个列表，由满足 ab 是图中的一条边的所有节点 b 组成。

老鼠从节点 1 开始，第一个出发；猫从节点 2 开始，第二个出发。在节点 0 处有一个洞。

在每个玩家的行动中，他们 必须 沿着图中与所在当前位置连通的一条边移动。例如，如果老鼠在节点 1 ，那么它必须移动到 graph[1] 中的任一节点。

此外，猫无法移动到洞中（节点 0）。

然后，游戏在出现以下三种情形之一时结束：

如果猫和老鼠出现在同一个节点，猫获胜。
如果老鼠到达洞中，老鼠获胜。
如果某一位置重复出现（即，玩家的位置和移动顺序都与上一次行动相同），游戏平局。
给你一张图 graph ，并假设两位玩家都都以最佳状态参与游戏：

如果老鼠获胜，则返回 1；
如果猫获胜，则返回 2；
如果平局，则返回 0 。
 
示例 1：


输入：graph = [[2,5],[3],[0,4,5],[1,4,5],[2,3],[0,2,3]]
输出：0

示例 2：


输入：graph = [[1,3],[0],[3],[0,2]]
输出：1
 

提示：

- 3 <= graph.length <= 50
- 1 <= graph[i].length < graph.length
- 0 <= graph[i][j] < graph.length
- graph[i][j] != i
- graph[i] 互不相同
- 猫和老鼠在游戏中总是可以移动

In [11]:
from typing import List
import queue

class Solution:
    def catMouseGame(self, graph: List[List[int]]) -> int:
        # 获取图的节点数
        N = len(graph)
        
        # 初始化DFA（确定性有限自动机）的子节点和父节点
        # dfa_children[k][i][j] 表示在状态k下，老鼠在i，猫在j时，下一步可能的状态
        # dfa_parents[k][i][j] 表示在状态k下，老鼠在i，猫在j时，上一步可能的状态
        dfa_children = [[[[] for j in range(N)] for i in range(N)] for k in range(2)]
        dfa_parents = [[[[] for j in range(N)] for i in range(N)] for k in range(2)]
        
        # 初始化DFA的结果数组，dfa_result[k][i][j] 表示在状态k下，老鼠在i，猫在j时的游戏结果
        # 0表示未确定，1表示老鼠赢，2表示猫赢
        dfa_result = [[[0 for j in range(N)] for i in range(N)] for k in range(2)]
        
        # 遍历所有可能的状态，构建DFA的子节点和父节点
        for i in range(N):
            if i == 0: continue  # 老鼠进洞，游戏结束，跳过
            for j in range(N):
                if i == j: continue  # 猫和老鼠在同一位置，游戏结束，跳过
                for nxi in graph[i]:  # 遍历老鼠的下一步可能位置
                    # 记录从状态0（老鼠移动）到状态1（猫移动）的父节点
                    dfa_parents[1][nxi][j].append([0, i, j])
                    # 记录从状态0（老鼠移动）到状态1（猫移动）的子节点
                    dfa_children[0][i][j].append([1, nxi, j])

        # 遍历所有可能的状态，构建DFA的子节点和父节点
        for j in range(N):
            for i in range(N):
                if i == 0: continue  # 老鼠进洞，游戏结束，跳过
                if i == j: continue  # 猫和老鼠在同一位置，游戏结束，跳过
                for nxj in graph[j]:  # 遍历猫的下一步可能位置
                    if nxj == 0: continue  # 猫不能进洞，跳过
                    # 记录从状态1（猫移动）到状态0（老鼠移动）的父节点
                    dfa_parents[0][i][nxj].append([1, i, j])
                    # 记录从状态1（猫移动）到状态0（老鼠移动）的子节点
                    dfa_children[1][i][j].append([0, i, nxj])

        # 初始化队列，用于广度优先搜索（BFS）
        que = queue.Queue()
        
        # 处理老鼠已经进洞的情况，老鼠赢
        for j in range(N):
            dfa_result[1][0][j] = 1  # 老鼠赢
            for node in dfa_parents[1][0][j]:
                que.put(node)  # 将父节点加入队列
        
        # 处理猫和老鼠在同一位置的情况，猫赢
        for k in range(1, N):
            dfa_result[0][k][k] = 2  # 猫赢
            dfa_result[1][k][k] = 2  # 猫赢
            for node in dfa_parents[0][k][k]:
                que.put(node)  # 将父节点加入队列
            for node in dfa_parents[1][k][k]:
                que.put(node)  # 将父节点加入队列

        # 开始广度优先搜索（BFS）
        while not que.empty():
            node = que.get()  # 取出队列中的节点
            if dfa_result[node[0]][node[1]][node[2]] != 0:
                continue  # 如果结果已经确定，跳过

            flag = True
            # 遍历当前节点的所有子节点
            for nx_node in dfa_children[node[0]][node[1]][node[2]]:
                # 如果子节点的结果是当前玩家的胜利条件
                if dfa_result[nx_node[0]][nx_node[1]][nx_node[2]] == 1 + node[0]:
                    dfa_result[node[0]][node[1]][node[2]] = 1 + node[0]  # 当前玩家赢
                    for p_node in dfa_parents[node[0]][node[1]][node[2]]:
                        que.put(p_node)  # 将父节点加入队列
                    flag = False
                    break
                # 如果子节点的结果未确定
                if dfa_result[nx_node[0]][nx_node[1]][nx_node[2]] == 0:
                    flag = False
            # 如果所有子节点都未导致当前玩家赢，则当前玩家输
            if flag:
                dfa_result[node[0]][node[1]][node[2]] = 2 - node[0]  # 对方玩家赢
                for p_node in dfa_parents[node[0]][node[1]][node[2]]:
                    que.put(p_node)  # 将父节点加入队列

        # 返回初始状态（老鼠在1，猫在2）的结果
        return dfa_result[0][1][2]
    
    

if __name__ == '__main__':
    sol = Solution()
    graph = [[5,7,9],[3,4,5,6],[3,4,5,8],[1,2,6,7],[1,2,5,7,9],[0,1,2,4,8],[1,3,7,8],[0,3,4,6,8],[2,5,6,7,9],[0,4,8]]
    print(sol.catMouseGame(graph))


1
